In [ ]:
# Magic functions -- Run Once
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import matplotlib.pyplot as plt
# Set default figure size
plt.rcParams["figure.figsize"] = (12, 5)

import warnings
# Ignore all warnings (This might not be a good idea)
warnings.filterwarnings('ignore')

# Move up one folder to reach the repo root
%cd ..

from utils.notebook.generic import full_width_notebook
# Set notebook to 100% width
full_width_notebook()

In [ ]:
from main import parse_args_string, prepare_for_task
from utils.notebook.generic import notebook_input_prompt
from utils.visualization import get_tagged_scene, get_tagged_scene_table_legend
from runner import custom_question_inference

data_version_name = "CLEAR_5k_5_inst_1024_win_50_overlap"
model_restore_path = "output/training/CLEAR_10k_20_inst_1024_win_50_overlap_reduction_original_rnn_256_40_epoch_stop_at_0.85_876944/2020-04-26_04h18"
#model_restore_path = "output/training/v3_fixed_5k_40_inst_1024_win_50_overlap_hpc-puget-necotis/latest"

data_root_path = "data"
config_path = "config/reduction/original_rnn_256.json"
#config_path = f"{model_restore_path}/config_raw_h5_input.json"
#config_path = "output/training/v3_fixed_5k_40_inst_1024_win_50_overlap_hpc-puget-necotis/latest/config_conv_input.json"
random_seed = 876944
image_height = 224
image_width = 224

# TODO : Retrieve params from {model_restore_path}
# TODO : What happen when processed with resnet ?

use_cpu = True

# Model weight restore
model_weight_path = f"{model_restore_path}/best/model.pt.tar"
model_dict_file_path = f"{model_restore_path}/dict.json"

# FIXME : Clear mean & std might be wrong (It is written in the config file)

arguments = (
    f"--notebook_model_inference --version_name {data_version_name} --config_path {config_path} --batch_size 4 "
    f"--random_seed {random_seed} --dict_folder questions "
    #f"--raw_img_resize_val {image_size} --no_feature_extractor --normalize_with_clear_stats "
    f"--film_model_weight_path {model_weight_path} --dict_file_path {model_dict_file_path} {'--use_cpu' if use_cpu else ''}"
)

#  
            
args = parse_args_string(arguments)
task_and_more, dataloaders, model_and_more = prepare_for_task(args)
print("Preparation done")
task, args, flags, paths, device = task_and_more
film_model_config, film_model, optimizer, loss_criterion, scheduler, tensorboard = model_and_more
datasets = {set_type:dloader.dataset for set_type, dloader in dataloaders.items()}

# Retrieve clear stats
import json
with open(f"{data_root_path}/{data_version_name}/clear_stats.json", 'r') as f:
    clear_stats = json.load(f)

# Print model summary
with open(f"{model_restore_path}/model_summary.txt", 'r') as f:
    print(f.read())


In [ ]:
# Retrieve random game
set_type = 'train'
random_game = datasets[set_type].get_random_game(return_game=True)
scene_id = random_game['scene_id']

In [ ]:
# Show scene & Example questions
from utils.notebook.inference import show_game_notebook_input

custom_question, custom_questions, legend = show_game_notebook_input(dataloaders[set_type], random_game, clear_stats,
                                                                     remove_image_padding=True)

In [ ]:
from utils.notebook.inference import do_custom_question_inference, show_gradcam

# MODIFY THIS TO GET GRADCAM FOR DIFFERENT GUESSES
gradcam_guess_id = 0

custom_game, top_preds = do_custom_question_inference(device, film_model, dataloaders[set_type], custom_question, scene_id, nb_top_pred=5)
display(legend)

In [ ]:
heatmaps = show_gradcam(device, film_model, dataloaders[set_type], custom_game, scene_id, guess_id=gradcam_guess_id, 
                        top_preds=top_preds, clear_stats=clear_stats, apply_relu=False)

In [ ]:
# Activations
